In [2]:
import numpy as np
import scipy
from scipy.ndimage import gaussian_filter1d
import sklearn
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture,BayesianGaussianMixture

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['image.cmap'] = 'Greys'
%matplotlib inline
import seaborn as sns

import sys,os,pdb,copy,pickle
from importlib import reload
import pynapple as nap

sys.path.append('/mnt/home/szheng/projects/seq_detection2/code')
sys.path.append('/mnt/home/szheng/projects/util_code')
sys.path.append('/mnt/home/szheng/projects/nmf_analysis')
sys.path.append('/mnt/home/szheng/projects/nmf_analysis/scripts')
sys.path.append('/mnt/home/szheng/projects/cluster_spikes')
import data_prep_new as dpn
import place_cell_analysis as pa
import plot_helper as ph
import preprocess as prep
import nmf_analysis as na
import nmf_plot as nmfp
reload(na)

import raster_new as rn
import data_prep_pyn as dpp
import database
db = database.db

import change_point_analysis as cpa
import change_point_plot as cpp

import place_field_analysis as pf
import pingouin as pg
import trial_correlates as tc
import tqdm
import misc

import switch_analysis_one_session as saos
reload(saos)

reload(misc)

67 sessions!
67 sessions!


<module 'misc' from '/mnt/home/szheng/projects/util_code/misc.py'>

In [3]:
figdir = '/mnt/home/szheng/ceph/place_variability/fig/cosyne'

In [4]:
db_sorted = db.query('owner=="roman"').sort_values('n_pyr_putative',ascending=False)
data_dir_full = db_sorted['data_dir_full'][0]

In [5]:
prep_res = dpp.load_spk_beh_df(data_dir_full,force_reload=False,extra_load={})
spk_beh_df=prep_res['spk_beh_df']

In [6]:
from collections import OrderedDict

In [8]:
db_sorted.loc['e16_3m2']

,data_dir_full,sess_name,animal_name.1,date,behavior,performance,ntrials,ngoodtrials,n_neurons,n_pyr_putative,owner,data_root
3,/mnt/home/szheng/ceph/ad/roman_data/e16/e16_3m...,e16_3m2_211213,e16_3m2,2021-12-13,['alternation' 'alternation'],0.703704,54,54,207,163,roman,roman_data
2,/mnt/home/szheng/ceph/ad/roman_data/e16/e16_3m...,e16_3m2_211212,e16_3m2,2021-12-12,['alternation' 'linearMaze'],0.821429,28,28,200,162,roman,roman_data
1,/mnt/home/szheng/ceph/ad/roman_data/e16/e16_3m...,e16_3m2_211211,e16_3m2,2021-12-11,['alternation' 'linearMaze'],0.870968,31,31,197,161,roman,roman_data
0,/mnt/home/szheng/ceph/ad/roman_data/e16/e16_3m...,e16_3m2_211210,e16_3m2,2021-12-10,alternation,0.760000,25,25,195,159,roman,roman_data


In [166]:

trial_cor_l['e13_26m1_210913']

v_mean      v_cv  idle_ratio  mean_fr_pyr  frac_active_pyr  \
         trial                                                                 
(0, 0.0) 1.0    8.591506  1.582938    0.422709     0.037013         0.822275   
         3.0    9.860801  1.428164    0.396802     0.039564         0.746445   
         6.0    7.641017  1.716568    0.431734     0.038548         0.791469   
         8.0    8.348290  1.545666    0.355908     0.037141         0.744076   
         10.0   7.978467  1.625535    0.408673     0.038832         0.798578   
...                  ...       ...         ...          ...              ...   
(0, 1.0) 62.0   5.253008  2.357105    0.564941     0.039094         0.872038   
         64.0   7.448366  1.675052    0.516509     0.039307         0.746445   
         66.0   8.478511  1.426783    0.398639     0.040336         0.770142   
         67.0   7.993331  1.236361    0.204148     0.040934         0.810427   
         69.0   8.445957  1.303829    0.182037     0.041538         0.793839   

                correct  correct_prev  
         trial                         
(0, 0.0) 1.0        1.0           NaN  
         3.0        1.0           1.0  
         6.0        1.0           0.0  
         8.0        1.0           1.0  
         10.0       1.0           1.0  
...                 ...           ...  
(0, 1.0) 62.0       1.0           1.0  
         64.0       1.0           1.0  
         66.0       1.0           1.0  
         67.0       0.0           1.0  
         69.0       1.0           1.0  

[70 rows x 7 columns]

In [181]:
reload(tc)
# trial_cor = tc.get_trial_correlates_wrapper(data_dir_full,dosave=True,save_fn='trial_correlates_vthresh.p',save_dir='trial_correlates',force_reload=True)




<module 'trial_correlates' from '/mnt/home/szheng/projects/util_code/trial_correlates.py'>

In [184]:
reload(tc)
failed_l = []
trial_cor_l = OrderedDict()
for ii in tqdm.tqdm(range(len(db_sorted['data_dir_full']))):
    ddf = db_sorted['data_dir_full'][ii]
    sess_name = db_sorted['sess_name'][ii]
    animal_name = db_sorted['animal_name.1'][ii]
    try:
        trial_cor = tc.get_trial_correlates_wrapper(ddf,dosave=True,save_fn='trial_correlates_vthresh.p',save_dir='trial_correlates',force_reload=True)
        trial_cor_l[(animal_name,sess_name)] = trial_cor
    except Exception as e:
        print(sess_name)
        print(e)
        failed_l.append(ddf)


  1%|▏         | 1/67 [00:10<11:29, 10.45s/it]

saved at /mnt/home/szheng/ceph/ad/roman_data/e13/e13_26m1/e13_26m1_210913/py_data/trial_correlates/trial_correlates_vthresh.p


  3%|▎         | 2/67 [00:22<12:25, 11.47s/it]

saved at /mnt/home/szheng/ceph/ad/roman_data/e15/e15_13f1/e15_13f1_220117/py_data/trial_correlates/trial_correlates_vthresh.p


  4%|▍         | 3/67 [00:36<13:21, 12.53s/it]

...

saved at /mnt/home/szheng/ceph/ad/roman_data/e16/e16_1f1/e16_1f1_200824/py_data/trial_correlates/trial_correlates_vthresh.p
load *ripples.events.mat failed


 99%|█████████▊| 66/67 [05:02<00:02,  2.64s/it]

e14_2m3_201017
'Column not found: v'
load *ripples.events.mat failed


100%|██████████| 67/67 [05:04<00:00,  4.55s/it]

e14_2m3_201019
'Column not found: v'


In [6]:
import pickle
savedatadir = '/mnt/home/szheng/ceph/place_variability/data'
# pickle.dump(trial_cor_l,open(os.path.join(savedatadir,'trial_corr_all_romsess.p'),'wb'))
trial_cor_l=pickle.load(open(os.path.join(savedatadir,'trial_corr_all_romsess.p'),'rb'))

In [5]:
# import multiprocessing as mp
# from multiprocessing import Pool

In [185]:
# data_dir_and_sess=list(db_sorted[['data_dir_full','sess_name']].itertuples(index=False,name=None))
data_dir_and_sess=list(db_sorted[['data_dir_full','animal_name.1','sess_name']].itertuples(index=False,name=None))

In [ ]:
res_to_save_data_detection_l = OrderedDict()
save_fn = 'shuffle_detection_vthresh_switch_res_switch_magnitude_only_fr_mean.p'
# for ddf, sess in data_dir_and_sess:
for ddf,ani, sess in data_dir_and_sess:
    res_to_save_data_detection_l[(ani,sess)] = saos.switch_analysis_one_session(ddf,place_field_res=None,force_reload=False,nrepeats=1000,dosave=False,save_fn=save_fn,load_only=True)
plt.close('all')

In [7]:
# load switch detection result
res_to_save_data_detection_l = pickle.load(open(os.path.join('/mnt/home/szheng/ceph/place_variability/data','switch_res_allromsess.p'),'rb'))


In [12]:
len(res_to_save_data_detection_l)

67

In [8]:
detection='avg'

res_to_save_data = list(res_to_save_data_detection_l.items())[0][1]
changes_df_pivoted=(res_to_save_data[detection]['changes_df']).stack(dropna=True).unstack(level=(2,3,4))
switch_on_pivoted = changes_df_pivoted==1
switch_on_all=switch_on_pivoted.groupby(level=(0,1,4)).sum()
switch_on_all_ratio = switch_on_pivoted.groupby(level=(0,1,4)).mean()

In [9]:
def get_on_off_ratio_all(res_to_save_data):
    '''
    res_to_save_data: no detection; 
    
    switch_all_ratio_d: df: (trialtype x trials) x ([1,-1] x pen x switch definition x switch threshold)
    '''
    changes_df_pivoted=(res_to_save_data['changes_df']).stack(dropna=True).unstack(level=(2,3,4))
    switch_all_ratio_d = {}
    for onoff in [1,-1]:
        switch_pivoted = changes_df_pivoted==onoff

        switch_all=switch_pivoted.groupby(level=(0,1,4)).sum()
        switch_all_ratio = switch_pivoted.groupby(level=(0,1,4)).mean()
        switch_all_ratio_d[onoff] = switch_all_ratio
    switch_all_ratio_d = pd.concat(switch_all_ratio_d,axis=1)
    return switch_all_ratio_d

In [10]:
def fix_corr_index(corr):

    max_index_len = np.max([len(x) if isinstance(x,tuple) else 0 for x in corr.index])
    index_corrected=[]
    for x in corr.index:
        if not isinstance(x, tuple):
            ind=tuple([x] + [None] * (max_index_len-1))
            index_corrected.append(ind)
        else:
            index_corrected.append(x)
    index_corrected = pd.MultiIndex.from_tuples(index_corrected)
    corr.index=index_corrected
    corr.columns=index_corrected
    
    return corr

def switch_corr_with_covar(switch_all_ratio_d,trial_correlates,stars=False,**kwargs):
    '''
    switch_all_ratio_d: see get_on_off_ratio_all
    trial_correlates: see tc.get_trial_correlates_wrapper
    '''
    df = pd.concat([switch_all_ratio_d.reset_index(drop=True),trial_correlates.reset_index(drop=True)],axis=1)
    corr = df.rcorr(stars=stars,**kwargs)
    corr = fix_corr_index(corr)
    index_corrected = corr.index
    corr_val_df = np.tril(corr,-1).astype(float) +np.tril(corr,-1).astype(float).T
    corr_val_df = pd.DataFrame(corr_val_df,index=index_corrected,columns=index_corrected)
    if not stars:
        corr_pval_df = np.triu(corr,1).astype(float) + np.triu(corr,1).astype(float).T
        corr_pval_df = pd.DataFrame(corr_pval_df,index=index_corrected,columns=index_corrected)
    else:
        corr_pval_df = None
    
    return corr_val_df, corr_pval_df, corr
    

In [11]:
def zscore(df):
    return (df-df.mean()) / df.std()

In [12]:
%%time
switch_all_ratio_allsess={}
corr_val_df_agg = {}
corr_pval_df_agg = {}
for (ani,sess), trial_cor in trial_cor_l.items():
    res_to_save_data = res_to_save_data_detection_l[ani,sess][detection]
    switch_all_ratio_d = get_on_off_ratio_all(res_to_save_data)
    switch_all_ratio_allsess[ani,sess] = switch_all_ratio_d
    corr_val_df_onesess,corr_pval_df_onesess,corr_onesess = switch_corr_with_covar(switch_all_ratio_d,trial_cor_l[ani,sess])
    corr_val_df_agg[ani,sess] = corr_val_df_onesess
    corr_pval_df_agg[ani,sess] = corr_pval_df_onesess
    
switch_all_ratio_allsess = pd.concat(switch_all_ratio_allsess,axis=0)
trial_cor_allsess=pd.concat(trial_cor_l.values(),axis=0)

corr_val_df, corr_pval_df, corr = switch_corr_with_covar(switch_all_ratio_allsess,trial_cor_allsess)

trial_cor_allsess_zscorewithinsess=pd.concat([zscore(trialcor) for trialcor in trial_cor_l.values()],axis=0)
corr_val_df_z, corr_pval_df_z, corr_z = switch_corr_with_covar(switch_all_ratio_allsess,trial_cor_allsess_zscorewithinsess)


/mnt/home/szheng/ceph/envs/jax/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


CPU times: user 3.79 s, sys: 17.8 ms, total: 3.81 s
Wall time: 3.84 s


In [212]:
pickle.dump(res_to_save_data_detection_l, open(os.path.join('/mnt/home/szheng/ceph/place_variability/data','switch_res_allromsess.p'),'wb'))





In [193]:
corr_val_df_agg = pd.concat(corr_val_df_agg,axis=0)
corr_pval_df_agg = pd.concat(corr_pval_df_agg,axis=0)

In [197]:
corr_pval_df_agg.groupby(level=(2,3,4,5)).median()

1                       \
                                         raw                        
                            switch_magnitude                        
                                         0.0    0.2    0.4    0.6   
-1 0.3 switch_magnitude 0.0            0.169  0.092  0.096  0.166   
                        0.2            0.169  0.092  0.096  0.166   
                        0.4            0.169  0.087  0.094  0.166   
                        0.6            0.186  0.099  0.112  0.210   
   0.5 switch_magnitude 0.0            0.329  0.201  0.153  0.302   
                        0.2            0.329  0.201  0.153  0.302   
                        0.4            0.329  0.201  0.153  0.302   
                        0.6            0.331  0.192  0.168  0.286   
   raw switch_magnitude 0.0            0.003  0.018  0.066  0.221   
                        0.2            0.015  0.044  0.089  0.192   
                        0.4            0.062  0.062  0.105  0.173   
                        0.6            0.117  0.084  0.079  0.147   
1  0.3 switch_magnitude 0.0            0.018  0.004  0.001  0.001   
                        0.2            0.018  0.004  0.001  0.001   
                        0.4            0.021  0.003  0.001  0.001   
                        0.6            0.046  0.008  0.001  0.000   
   0.5 switch_magnitude 0.0            0.209  0.171  0.230  0.169   
                        0.2            0.209  0.171  0.230  0.169   
                        0.4            0.209  0.171  0.230  0.169   
                        0.6            0.220  0.219  0.237  0.172   
   raw switch_magnitude 0.0            0.000  0.000  0.000  0.005   
                        0.2            0.000  0.000  0.000  0.000   
                        0.4            0.000  0.000  0.000  0.000   
                        0.6            0.005  0.000  0.000  0.000   

                                                                   \
                                         0.3                        
                            switch_magnitude                        
                                         0.0    0.2    0.4    0.6   
-1 0.3 switch_magnitude 0.0            0.420  0.420  0.453  0.314   
                        0.2            0.420  0.420  0.453  0.314   
                        0.4            0.420  0.420  0.433  0.314   
                        0.6            0.365  0.365  0.364  0.256   
   0.5 switch_magnitude 0.0            0.413  0.413  0.426  0.302   
                        0.2            0.413  0.413  0.426  0.302   
                        0.4            0.413  0.413  0.426  0.302   
                        0.6            0.414  0.414  0.421  0.315   
   raw switch_magnitude 0.0            0.312  0.312  0.312  0.350   
                        0.2            0.241  0.241  0.259  0.276   
                        0.4            0.184  0.184  0.184  0.236   
                        0.6            0.257  0.257  0.257  0.266   
1  0.3 switch_magnitude 0.0            0.000  0.000  0.000  0.000   
                        0.2            0.000  0.000  0.000  0.000   
                        0.4            0.000  0.000  0.000  0.000   
                        0.6            0.000  0.000  0.000  0.000   
   0.5 switch_magnitude 0.0            0.001  0.001  0.001  0.004   
                        0.2            0.001  0.001  0.001  0.004   
                        0.4            0.001  0.001  0.001  0.004   
                        0.6            0.002  0.002  0.002  0.002   
   raw switch_magnitude 0.0            0.018  0.018  0.021  0.046   
                        0.2            0.004  0.004  0.003  0.008   
                        0.4            0.001  0.001  0.001  0.001   
                        0.6            0.001  0.001  0.001  0.000   

                                                     ...               -1  \
                                         0.5         ...              0.5   
                       

In [201]:
corr_pval_df_agg.loc[(slice(None),slice(None),'ripple_rate'),(1,0.5,'switch_magnitude',0.2)]

e13_26m1  e13_26m1_210913  ripple_rate  NaN  NaN  NaN    0.042
e15_13f1  e15_13f1_220117  ripple_rate  NaN  NaN  NaN    0.589
          e15_13f1_220118  ripple_rate  NaN  NaN  NaN    0.241
          e15_13f1_220119  ripple_rate  NaN  NaN  NaN    0.750
          e15_13f1_220120  ripple_rate  NaN  NaN  NaN    0.234
e13_26m1  e13_26m1_211019  ripple_rate  NaN  NaN  NaN    0.100
e13_16f1  e13_16f1_210328  ripple_rate  NaN  NaN  NaN    0.132
e13_26m1  e13_26m1_211116  ripple_rate  NaN  NaN  NaN    0.284
e16_3m2   e16_3m2_211213   ripple_rate  NaN  NaN  NaN    0.836
e16_3m1   e16_3m1_210111   ripple_rate  NaN  NaN  NaN    0.010
e16_3m2   e16_3m2_211212   ripple_rate  NaN  NaN  NaN    0.034
          e16_3m2_211211   ripple_rate  NaN  NaN  NaN    0.140
e16_3m1   e16_3m1_210203   ripple_rate  NaN  NaN  NaN    0.968
e16_3m2   e16_3m2_211210   ripple_rate  NaN  NaN  NaN    0.732
e13_26m1  e13_26m1_211119  ripple_rate  NaN  NaN  NaN    0.850
          e13_26m1_211124  ripple_rate  NaN  NaN  NaN  

In [207]:
trial_cor_l['e13_26m1','e13_26m1_210913'].loc[0,0].loc[8]

v_mean             8.348290
v_cv               1.545666
idle_ratio         0.355908
mean_fr_pyr        0.037141
frac_active_pyr    0.744076
correct            1.000000
correct_prev       1.000000
ripple_count       0.000000
ripple_rate        0.000000
Name: 8.0, dtype: float64

In [145]:
db.loc[db['sess_name']=="e13_16f1_210319"]

,,data_dir_full,sess_name,animal_name.1,date,behavior,performance,ntrials,ngoodtrials,n_neurons,n_pyr_putative,owner,data_root
animal_name,,,,,,,,,,,,,
e13_16f1,5,/mnt/home/szheng/ceph/ad/roman_data/e13/e13_16...,e13_16f1_210319,e13_16f1,2021-03-19,alternation,0.76,25,25,120,95,roman,roman_data


In [132]:
corr_val_df_agg.groupby(level=(1,2,3,4)).median()

1                       \
                                         raw                        
                            switch_magnitude                        
                                         0.0    0.2    0.4    0.6   
-1 0.3 switch_magnitude 0.0            0.286  0.333  0.319  0.256   
                        0.2            0.286  0.333  0.319  0.256   
                        0.4            0.285  0.343  0.333  0.256   
                        0.6            0.267  0.336  0.320  0.269   
   0.5 switch_magnitude 0.0            0.211  0.275  0.274  0.225   
                        0.2            0.211  0.275  0.274  0.225   
                        0.4            0.211  0.275  0.274  0.225   
                        0.6            0.200  0.276  0.263  0.231   
   raw switch_magnitude 0.0            0.610  0.454  0.327  0.238   
                        0.2            0.511  0.411  0.341  0.271   
                        0.4            0.383  0.352  0.329  0.264   
                        0.6            0.300  0.355  0.356  0.301   
1  0.3 switch_magnitude 0.0            0.473  0.565  0.644  0.641   
                        0.2            0.473  0.565  0.644  0.641   
                        0.4            0.484  0.565  0.632  0.641   
                        0.6            0.424  0.530  0.632  0.651   
   0.5 switch_magnitude 0.0            0.266  0.268  0.253  0.281   
                        0.2            0.266  0.268  0.253  0.281   
                        0.4            0.266  0.268  0.253  0.282   
                        0.6            0.255  0.268  0.260  0.282   
   raw switch_magnitude 0.0            0.000  0.909  0.754  0.567   
                        0.2            0.909  0.000  0.875  0.699   
                        0.4            0.754  0.875  0.000  0.841   
                        0.6            0.567  0.699  0.841  0.000   

                                                                   \
                                         0.3                        
                            switch_magnitude                        
                                         0.0    0.2    0.4    0.6   
-1 0.3 switch_magnitude 0.0            0.162  0.162  0.162  0.195   
                        0.2            0.162  0.162  0.162  0.195   
                        0.4            0.162  0.162  0.162  0.195   
                        0.6            0.204  0.204  0.204  0.244   
   0.5 switch_magnitude 0.0            0.158  0.158  0.158  0.174   
                        0.2            0.158  0.158  0.158  0.174   
                        0.4            0.158  0.158  0.158  0.174   
                        0.6            0.158  0.158  0.158  0.198   
   raw switch_magnitude 0.0            0.234  0.234  0.234  0.195   
                        0.2            0.230  0.230  0.230  0.239   
                        0.4            0.273  0.273  0.262  0.273   
                        0.6            0.229  0.229  0.229  0.243   
1  0.3 switch_magnitude 0.0            0.000  1.000  1.000  0.935   
                        0.2            1.000  0.000  1.000  0.935   
                        0.4            1.000  1.000  0.000  0.937   
                        0.6            0.935  0.935  0.937  0.000   
   0.5 switch_magnitude 0.0            0.625  0.625  0.625  0.560   
                        0.2            0.625  0.625  0.625  0.560   
                        0.4            0.625  0.625  0.625  0.560   
                        0.6            0.600  0.600  0.600  0.594   
   raw switch_magnitude 0.0            0.473  0.473  0.484  0.424   
                        0.2            0.565  0.565  0.565  0.530   
                        0.4            0.644  0.644  0.632  0.632   
                        0.6            0.641  0.641  0.641  0.651   

                                                     ...               -1  \
                                         0.5         ...              0.5   
                       

In [122]:
corr_val_df

1                \
                                                      raw                 
                                         switch_magnitude                 
                                                      0.0    0.2    0.4   
1               raw switch_magnitude 0.0            0.000  0.879  0.696   
                                     0.2            0.879  0.000  0.864   
                                     0.4            0.696  0.864  0.000   
                                     0.6            0.513  0.683  0.852   
                0.3 switch_magnitude 0.0            0.442  0.544  0.630   
                                     0.2            0.442  0.544  0.630   
                                     0.4            0.438  0.542  0.631   
                                     0.6            0.389  0.501  0.609   
                0.5 switch_magnitude 0.0            0.248  0.270  0.274   
                                     0.2            0.248  0.270  0.274   
                                     0.4            0.248  0.270  0.274   
                                     0.6            0.234  0.264  0.274   
-1              raw switch_magnitude 0.0            0.500  0.348  0.203   
                                     0.2            0.383  0.313  0.239   
                                     0.4            0.262  0.274  0.248   
                                     0.6            0.188  0.242  0.256   
                0.3 switch_magnitude 0.0            0.224  0.260  0.259   
                                     0.2            0.224  0.260  0.259   
                                     0.4            0.223  0.263  0.263   
                                     0.6            0.207  0.263  0.267   
                0.5 switch_magnitude 0.0            0.159  0.190  0.183   
                                     0.2            0.159  0.190  0.183   
                                     0.4            0.158  0.191  0.184   
                                     0.6            0.153  0.196  0.194   
v_mean          NaN NaN              NaN            0.004 -0.032 -0.033   
v_cv            NaN NaN              NaN           -0.011 -0.007 -0.006   
idle_ratio      NaN NaN              NaN            0.010  0.015  0.003   
mean_fr_pyr     NaN NaN              NaN            0.004  0.047  0.059   
frac_active_pyr NaN NaN              NaN            0.011  0.035  0.027   
correct         NaN NaN              NaN           -0.020  0.015  0.000   
correct_prev    NaN NaN              NaN           -0.007 -0.028 -0.010   

                                                                         \
                                                             0.3          
                                                switch_magnitude          
                                            0.6              0.0    0.2   
1               raw switch_magnitude 0.0  0.513            0.442  0.442   
                                     0.2  0.683            0.544  0.544   
                                     0.4  0.852            0.630  0.630   
                                     0.6  0.000            0.653  0.653   
                0.3 switch_magnitude 0.0  0.653            0.000  1.000   
                                     0.2  0.653            1.000  0.000   
                                     0.4  0.655            0.997  0.997   
                                     0.6  0.663            0.930  0.930   
                0.5 switch_magnitude 0.0  0.270            0.613  0.613   
                                     0.2  0.270            0.613  0.613   
                                     0.4  0.270            0.614  0.614   
                                     0.6  0.287            0.592  0.592   
-1              raw switch_magnitude 0.0  0.127            0.191  0.191   
                                     0.2  0.197            0.220  0.220   
                                     0.4  0.235            0.244  0.244   
     

<function __main__.fix_corr_index(corr)>

In [94]:
covar_l = ['mean_fr_pyr','frac_active_pyr']
pd.concat([corr_pval_df.loc[covar_l,:].T,corr_val_df.loc[covar_l,:].T],axis=1)


mean_fr_pyr frac_active_pyr  \
                                                 NaN             NaN   
                                                 NaN             NaN   
                                                 NaN             NaN   
1               raw switch_magnitude 0.0       0.875           0.693   
                                     0.2       0.081           0.197   
                                     0.4       0.028           0.322   
                                     0.6       0.245           0.336   
                0.3 switch_magnitude 0.0       0.219           0.881   
                                     0.2       0.219           0.881   
                                     0.4       0.173           0.989   
                                     0.6       0.105           0.493   
                0.5 switch_magnitude 0.0       0.739           0.955   
                                     0.2       0.739           0.955   
                                     0.4       0.737           0.952   
                                     0.6       0.524           0.646   
-1              raw switch_magnitude 0.0       0.911           0.946   
                                     0.2       0.477           0.538   
                                     0.4       0.335           0.059   
                                     0.6       0.280           0.320   
                0.3 switch_magnitude 0.0       0.148           0.280   
                                     0.2       0.148           0.280   
                                     0.4       0.100           0.249   
                                     0.6       0.071           0.274   
                0.5 switch_magnitude 0.0       0.154           0.225   
                                     0.2       0.154           0.225   
                                     0.4       0.147           0.209   
                                     0.6       0.072           0.165   
v_mean          NaN NaN              NaN       0.000           0.000   
v_cv            NaN NaN              NaN       0.388           0.289   
idle_ratio      NaN NaN              NaN       0.001           0.000   
mean_fr_pyr     NaN NaN              NaN       0.000           0.000   
frac_active_pyr NaN NaN              NaN       0.000           0.000   
correct         NaN NaN              NaN       0.008           0.030   
correct_prev    NaN NaN              NaN       0.018           0.000   

                                         mean_fr_pyr frac_active_pyr  
                                                 NaN             NaN  
                                                 NaN             NaN  
                                                 NaN             NaN  
1               raw switch_magnitude 0.0       0.004           0.011  
                                     0.2       0.047           0.035  
                                     0.4       0.059           0.027  
                                     0.6       0.031           0.026  
                0.3 switch_magnitude 0.0       0.033          -0.004  
                                     0.2       0.033          -0.004  
                                     0.4       0.037           0.000  
                                     0.6       0.044           0.018  
                0.5 switch_magnitude 0.0       0.009           0.002  
                                     0.2       0.009           0.002  
                                     0.4       0.009           0.002  
                                     0.6       0.017           0.012  
-1              raw switch_magnitude 0.0       0.003          -0.002  
                                     0.2       0.019           0.017  
                                     0.4       0.026           0.051  
                                     0.6       0.029           0.027  
                0.3 switch_magnitude 0.0       0.039           0.029  
                             

In [23]:
switch_on_all

raw                          0.3              \
       switch_magnitude             switch_magnitude               
                    0.0 0.2 0.4 0.6              0.0 0.2 0.4 0.6   
0 0 0                 0   0   0   0                0   0   0   0   
    1               178  76  27  11                1   1   1   1   
    2               194  74  27  10                9   9   9   9   
    3               178  72  25   7               15  15  15  15   
    4               187  85  32  14               18  18  18  17   
...                 ...  ..  ..  ..              ...  ..  ..  ..   
  1 32              181  81  44  19               18  18  18  16   
    33              158  68  22  13                7   7   6   3   
    34              161  60  22   8                6   6   5   3   
    35              192  95  44  23               15  15  15  12   
    36              165  74  27   8               13  13  13  13   

                    0.5              
       switch_magnitude              
                    0.0 0.2 0.4 0.6  
0 0 0                 0   0   0   0  
    1                 0   0   0   0  
    2                 2   2   2   2  
    3                 8   8   8   8  
    4                10  10  10  10  
...                 ...  ..  ..  ..  
  1 32                8   8   8   6  
    33                1   1   1   1  
    34                1   1   1   1  
    35                5   5   5   5  
    36                6   6   6   6  

[70 rows x 12 columns]

In [19]:
res_to_save_data_detection_l[0]['avg']['changes_df']

0   1   2   3   4   5   6   7   8   9   \
0 0 raw switch_magnitude 0.0 3   0   0   1  -1  -1   1  -1   0   0   0   1   
                             27  0   0  -1   1  -1   1  -1   1  -1  -1   1   
                             29  0   0   1   1  -1  -1   1  -1  -1   1  -1   
                                 1   0  -1   1  -1  -1   1  -1   1   1   1   
                                 2   0   1  -1  -1   1  -1   1  -1   1   1   
...                                 ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   
  1 0.5 switch_magnitude 0.6 463 0   0   0   0   0   0   0   0   0   0   0   
                             465 0   0   0   0   0   0   0   0   0   0   0   
                             475 0   0   0   0   0   0   0   0   0   0   0   
                             476 0   0   0   0   0   0   0   0   0  -1   0   
                             480 0   0   0   0   0   0   0   0   0   0   0   

                                    ...  27  28  29  30  31  32   33   34  \
0 0 raw switch_magnitude 0.0 3   0  ...  -1   1  -1  -1  -1   1  NaN  NaN   
                             27  0  ...  -1   1   1  -1   1  -1  NaN  NaN   
                             29  0  ...   1  -1  -1   1  -1   1  NaN  NaN   
                                 1  ...   1  -1   1  -1   1   1  NaN  NaN   
                                 2  ...  -1  -1   1   1  -1   1  NaN  NaN   
...                                 ...  ..  ..  ..  ..  ..  ..  ...  ...   
  1 0.5 switch_magnitude 0.6 463 0  ...   0   0   0   0   0   0  0.0  0.0   
                             465 0  ...   0   0   0   0   0   0  0.0  0.0   
                             475 0  ...   0   0   0   0   0   0  0.0  0.0   
                             476 0  ...   0   0   0   0   0   0  0.0  0.0   
                             480 0  ...   0   0   0   0   0   0  0.0  0.0   

                                     35   36  
0 0 raw switch_magnitude 0.0 3   0  NaN  NaN  
                             27  0  NaN  NaN  
                             29  0  NaN  NaN  
                                 1  NaN  NaN  
                                 2  NaN  NaN  
...                                 ...  ...  
  1 0.5 switch_magnitude 0.6 463 0  0.0  0.0  
                             465 0  0.0  0.0  
                             475 0  0.0  0.0  
                             476 0  0.0  0.0  
                             480 0  0.0  0.0  

[9144 rows x 37 columns]